# Consistent Bayes: Some Motivating Examples
---

Copyright 2017-2018 Michael Pilosov


### Import Libraries
_tested with python 3.6 on 01/26/18_

In [1]:
# Mathematics and Plotting
from HelperFuns import * # pyplot wrapper functions useful for visualizations, numpy, scipy, etc.
%matplotlib inline
plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = 10, 5
from cbayes import sample, solve, distributions
# Interactivity
from ipywidgets import *

---
## Formulating the Inverse Problem
---
### Prior Information/Assumptions

* We assume that the true value $\lambda_0$ belongs to a parameter space $\Lambda$.


* Much like in the classical statistical Bayesian framework, we begin by encapsulating our pre-existing beliefs about our parameters in a distribution in a prior distribution on $\Lambda$, $\pi^{prior}_\Lambda$

### The Observed Density

* The observed density represents the uncertainty in an observation of a measurable quantity of interest map that takes input parameters and produces a vector in $\mathbb{R}^d$.


* While there are problem scenarios you can posit where the observed density corresponds to a normalized likelihood function from the statistical Bayesian approach, the quantity of interest may not necessarily just be the uncertainty in the measurement data. 


* If the quantity of interest is indeed a single direct measurement, then the likelihood is the observed. For example, for some true parameter $\lambda_0$ and model $u(\lambda, t)$, suppose your quantity of interest is defined as a single evaluation at some time $t_0$. The measurement uncertainty contained in that one measurement would constitute your observed density. 


* However, if we have a collection of observations, such as at $t_0, t_2, \dots, t_K$,  each of which is polluted by normally distributed noise, a common thing to do from Bayesian and Frequentist statistics would be to minimize the mean-or sum-squared error. If the sum squared error (SSE) is what we treat as our quantity of interest, the observed density on $\mathcal{D}$, denoted by $\pi^{obs}_{\mathcal{D}}(d)$, would be given by a $\chi^2_{K+1}$ distribution.


### The Posterior Density

* Let $\pi^{O(prior)}_{\mathcal{D}}(d)$ denote the push-forward of the prior density onto $\mathcal{D}$. Then, the posterior density on $\Lambda$ is given by

$$
    \pi^{post}_\Lambda(\lambda) := \pi^{prior}_\Lambda(\lambda)\frac{\pi^{obs}_{\mathcal{D}}(Q(\lambda))}{\pi^{O(prior)}_{\mathcal{D}}(Q(\lambda))}
$$

---
## Define your Map
_ Choose from one of the following example options, feel free to add your own _ 

$O_1(\lambda) = \sum_{i=1}^n \lambda_i$  

$O_2(\lambda) = \lbrace \lambda_0,\;\; \lambda_0 - \lambda_1 \rbrace$ 

$O_3(\lambda) = \lbrace \lambda_0,\;\; \lambda_0 - \lambda_1, \; \;\lambda_1^2 \rbrace$

$O_4(\lambda) = (1-x)^2 + (y - x^2)^2$  ( This is the [Rosenbrock Function](https://en.wikipedia.org/wiki/Rosenbrock_function) with $a=1$ and $b=100$. )

In [2]:
PtO_fun_choice = 4

def fun1(lam): # sum all params
    return np.sum(lam,axis=1)

def fun2(lam): # pull two params, linear combination.
    return np.array([ lam[:,0] ,lam[:,0]-lam[:,1]]).transpose()

def fun3(lam): # pull two params, linear combination.
    return np.array([ lam[:,0] ,lam[:,0]-lam[:,1], lam[:,1]**2 ]).transpose()

def rosenbrock(lam):
     return (1.-lam[:,0])**2 + 100*(lam[:,1]-lam[:,0]**2.)**2
    
if PtO_fun_choice == 1:
    PtO_fun = fun1
elif PtO_fun_choice == 2:
    PtO_fun = fun2
elif PtO_fun_choice == 3:
    PtO_fun = fun3
elif PtO_fun_choice == 4:
    PtO_fun = rosenbrock
else:
    raise( ValueError('Specify Proper PtO choice!') )

--- 

# Sample from $\Lambda$
_Here we implement uniform random priors on the unit hypercube_

In [3]:
input_dim = 2 # Specify input space dimension (n)
num_samples = int(1E3) # number of input samples (N)
s_set = sample.sample_set(size=(num_samples, input_dim))

if PtO_fun_choice == 1:
    s_set.set_dist('normal', {'loc': 0, 'scale': 1})
elif PtO_fun_choice == 2:
    s_set.set_dist('normal', {'loc': 0, 'scale': 1})
elif PtO_fun_choice == 3:
    s_set.set_dist('normal', {'loc': 0, 'scale': 1})
elif PtO_fun_choice == 4: # rosenbrock
    s_set.set_dist('uniform', {'loc': [-1, -1], 'scale': [2, 1]})
                   
s_set.generate_samples()

lam = s_set.samples # create a pointer for ease of reference later with plotting.

### Visualize Prior 

In [4]:
widgets.interactive(pltdata, data = fixed(lam), inds = fixed(None), 
                    N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
                    eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="orange"),
                    view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
                    view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=1), IntSlider(value=1, continuous_update=False, description='view_dim_2', max=1), IntSlider(value=500, continuous_update=False, description='N', max=5000, min=100, step=100), Text(value='orange', description='color'), Checkbox(value=False, description='svd'), Output()), _dom_classes=('widget-interact',))

---
# Compute Data Space $O(\Lambda) = \mathcal{D}$ 

Format: `(n_dims, n_samples)`  

In [5]:
p_set = sample.map_samples_and_create_problem(s_set, PtO_fun)
D = p_set.output.samples

# this is how we handle trying to infer the dimension based on what the map put out.
try:
    output_dim = D.shape[1] # if your function was coded correctly, you should have an (n, d) data space.
except IndexError:
    print(Warning("Warning: Your map might be returning the wrong dimensional data."))
    try:
       output_dim = D.shape[0] 
    except IndexError:
        print(Warning("Warning: Guessing it's 1-dimensional."))
        output_dim = 1
print('dimensions :  lambda = '+str(lam.shape)+'   D = '+str(D.shape) )

dimensions :  lambda = (1000, 2)   D = (1000, 1)


# Compute Push-Forward of the Prior $P_{O(\Lambda)}$
_ ... i.e. Characterize the Data Space_

In [6]:
# Interactive Marginal Visualization
p_set.compute_pushforward_dist()
pf_dist = p_set.pushforward_dist

In [7]:
widgets.interactive(pltdata, data = fixed(pf_dist), inds = fixed(None), 
        N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="brown"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=0), IntSlider(value=0, continuous_update=False, description='view_dim_2', max=0), IntSlider(value=500, continuous_update=False, description='N', max=5000, min=100, step=100), Text(value='brown', description='color'), Output()), _dom_classes=('widget-interact',))

# Define Observed Probability Measure $P_\mathcal{D}$

In [8]:
if PtO_fun_choice == 4:
    p_set.set_observed_dist('normal', {'loc': 100, 'scale': 12}) # FOR ROSENBROCK

elif PtO_fun_choice == 3:
#         p_set.set_observed_dist('normal', {'loc':[0, 0, 0], 'scale':[0.5, 0.25, 1]}) # better for function choice = 2
    p_set.set_observed_dist('uniform', {'loc':[-0.5, -0.5, -0.5], 'scale':[1, 1, 1]}) # better for function choice = 2

elif PtO_fun_choice == 2:
    p_set.set_observed_dist('uniform', {'loc':[0, 0], 'scale':[1, 0.25]}) # default is normal based on the data space # for function choice = 1

elif PtO_fun_choice == 1:
    p_set.set_observed_dist('uni', {'loc':0, 'scale':0.3}) # default is normal based on the data space # for function choice = 1

obs_dist = p_set.observed_dist # this is define a pointer for ease of reference.

widgets.interactive(pltdata, data = fixed(obs_dist), inds = fixed(None), 
        N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="wine"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=0), IntSlider(value=0, continuous_update=False, description='view_dim_2', max=0), IntSlider(value=500, continuous_update=False, description='N', max=5000, min=100, step=100), Text(value='wine', description='color'), Output()), _dom_classes=('widget-interact',))

---

At this point we have performed the computations we need to. We have evaluated the input points through our map and performed a KDE on them. It would be useful at this point to save this object and/or its evaluation at every point in the data space for later re-use. Doing so here would be an appropriate place. 

--- 

# Accept/Reject Sampling of Posterior

Since we have already used the samples in our prior to compute the pushforward density, we can re-use these with an accept/reject algorithm to get a set of samples generated from the posterior according to the solution of the stochastic inverse problem as outlined in the Consistent Bayes formulation. 

In [9]:
p_set.set_ratio()
eta_r = p_set.ratio
solve.problem(p_set)

In [10]:
accept_inds = p_set.accept_inds
lam_accept = p_set.input.samples[accept_inds,:]
num_accept = len(accept_inds)
print('Number accepted: %d = %2.2f%%'%(num_accept, 100*np.float(num_accept)/num_samples))

Number accepted: 145 = 14.50%


## Visualize Posterior Density
### (Visualize Accept/Reject Samples)


In [11]:
widgets.interactive(pltdata, data = fixed(lam), inds = fixed(accept_inds), 
        N = widgets.IntSlider(value=num_accept/2, min = 2, max=num_accept+1, step=1, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="orange"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False))

# You will visualize the accepted samples in a subset of size N of the input samples. 
# This is mostly for faster plotting, but also so you can see the progression of accepted sampling in the algorithm.


interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=1), IntSlider(value=1, continuous_update=False, description='view_dim_2', max=1), IntSlider(value=72, continuous_update=False, description='N', max=146, min=2), Text(value='orange', description='color'), Checkbox(value=False, description='svd'), Output()), _dom_classes=('widget-interact',))

---
# Now what? 

Well, we can...

## _Visualize the Quality of our SIP Solution by Comparing it to the Observed_
_We compare the push-forward of the posterior using accepted samples against the observed density_  
_(SIP = Stochastic Inverse Problem)_
### Observed:

In [12]:
widgets.interactive(pltdata, data = fixed(obs_dist), inds = fixed(None), 
        N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="wine"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))

interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=0), IntSlider(value=0, continuous_update=False, description='view_dim_2', max=0), IntSlider(value=500, continuous_update=False, description='N', max=5000, min=100, step=100), Text(value='wine', description='color'), Output()), _dom_classes=('widget-interact',))

## Pushforward of Posterior:

In [13]:
widgets.interactive(pltdata, data = fixed(D), inds = fixed(accept_inds), 
        N = widgets.IntSlider(value=num_accept/2, min = 2, max=num_accept-1, step=1, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="eggplant"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=0), IntSlider(value=0, continuous_update=False, description='view_dim_2', max=0), IntSlider(value=72, continuous_update=False, description='N', max=144, min=2), Text(value='eggplant', description='color'), Output()), _dom_classes=('widget-interact',))

# Using the Posterior in Other Ways...

## Utilizing a KDE of the Posterior
### _Use KDE of posterior to generate new samples_

In [14]:
post_dens_kde = distributions.gkde(lam_accept)
n = 5000 # number of new samples to generate using the KDE
new_inputs = post_dens_kde.rvs(n)

widgets.interactive(pltdata, data = fixed(new_inputs), inds = fixed(None), 
        N = widgets.IntSlider(value=1000, min = 1, max=5000, step=100, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="tiffany blue"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False))

# plt.show()

interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=1), IntSlider(value=1, continuous_update=False, description='view_dim_2', max=1), IntSlider(value=1000, continuous_update=False, description='N', max=5000, min=1, step=100), Text(value='tiffany blue', description='color'), Checkbox(value=False, description='svd'), Output()), _dom_classes=('widget-interact',))

These don't do an excellent job of simulating the accepted samples per say. 

### _Perhaps these do a decent job resolving the observed density? Let's push-forward the samples from the KDE of the Posterior through our map and see:_

In [15]:
new_outputs = PtO_fun(new_inputs).reshape((n, output_dim))
widgets.interactive(pltdata, data = fixed(new_outputs), inds = fixed(None), 
        N = widgets.IntSlider(value=1000, min = 1, max=5000, step=100, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="cyan"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=0), IntSlider(value=0, continuous_update=False, description='view_dim_2', max=0), IntSlider(value=1000, continuous_update=False, description='N', max=5000, min=1, step=100), Text(value='cyan', description='color'), Checkbox(value=False, description='svd'), Output()), _dom_classes=('widget-interact',))

### _More likely, we would use the KDE instead to generate proposal samples for accept/reject._

In [16]:
# This also outlines the tricks we have to do to handle multi-dimensional parametric distributions. 
ob = obs_dist.pdf(new_outputs).prod(axis=1).reshape(n)
pf = pf_dist.pdf(new_outputs).reshape(n)
new_ratios = ob/pf
new_inds = solve.perform_accept_reject(new_inputs, new_ratios)
new_accepted_samples = new_inputs[new_inds,:]
new_num_accept = new_accepted_samples.shape[0]
print("Num. Accepted = %d out of %d (%2.2f%%)"%(new_num_accept, n, 100*new_num_accept/n))

Num. Accepted = 1156 out of 5000 (23.12%)


In [17]:
widgets.interactive(pltdata, data = fixed(new_inputs), inds = fixed(new_inds), 
        N = widgets.IntSlider(value=new_num_accept/2, min = 1, max=new_num_accept, step=1, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="tiffany blue"),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='view_dim_1', max=1), IntSlider(value=1, continuous_update=False, description='view_dim_2', max=1), IntSlider(value=578, continuous_update=False, description='N', max=1156, min=1), Text(value='tiffany blue', description='color'), Checkbox(value=False, description='svd'), Output()), _dom_classes=('widget-interact',))